In [15]:
import pandas as pd
import sqlite3
import os

In [16]:
df = pd.read_csv('customer_behavior_clean.csv')

In [17]:
os.makedirs('data/database',exist_ok=True)

conn = sqlite3.connect('data/database/customer_behavior.db')

df.to_sql(
    name='customer',
    con=conn,
    if_exists='replace',
    index=False
)

3900

In [ ]:
# Varify that dataframe is in a SQL database
pd.read_sql("SELECT * FROM customer LIMIT 5;", conn)


,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,Middle-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,Middle-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,Middle-aged,365


In [19]:
#Total Revenue by gender

pd.read_sql("""
    select gender, sum(purchase_amount) as revenue
    from customer
    group by gender
""",conn)

,gender,revenue
0,Female,75191
1,Male,157890


In [20]:
# Customers that used discount code and spent more than the average purchase amount
pd.read_sql("""
    select customer_id, purchase_amount
    from customer
    where discount_applied = 'Yes' and purchase_amount > (select avg(purchase_amount) from customer)
""",conn)

,customer_id,purchase_amount
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97
...,...,...
834,1667,64
835,1671,73
836,1673,73
837,1674,62


In [28]:
# Top 5 products by customer review
pd.read_sql("""
select item_purchased, ROUND(AVG(CAST(review_rating as REAL)), 2) as "Average Product Rating"
from customer
group by item_purchased
order by AVG (review_rating) desc
limit 5;
""",conn)

,item_purchased,Average Product Rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,Skirt,3.78


In [31]:
# Average puchase by Standard and Express shipping
pd.read_sql("""
select shipping_type,
round(avg(purchase_amount), 2) as average_purchase_amount
from customer
where shipping_type in ('Standard','Express')
group by shipping_type
""",conn)

,shipping_type,average_purchase_amount
0,Express,60.48
1,Standard,58.46


In [22]:
# Average spend and total revenue based on subscription status
pd.read_sql("""
select subscription_status, 
count(customer_id) as total_customer, 
round(avg(purchase_amount), 2) as avg_spend, 
round(sum(purchase_amount), 2) as total_rev
from customer
group by subscription_status
order by total_rev, avg_spend desc;
""",conn)

,subscription_status,total_customer,avg_spend,total_rev
0,Yes,1053,59.49,62645.0
1,No,2847,59.87,170436.0


In [23]:
# Top 5 products that have the highest precentage of purchases with discounts applied
pd.read_sql("""
select item_purchased,
round(100 * sum(case when discount_applied = 'Yes' then 1 else 0 end)/count(*), 2) as discount_rate
from customer
group by item_purchased
order by discount_rate desc
limit 5;
""",conn)

,item_purchased,discount_rate
0,Hat,50.0
1,Sneakers,49.0
2,Coat,49.0
3,Sweater,48.0
4,Pants,47.0


In [33]:
# Segment customer into New, Returning, and Loyal based on their total number of previous purchases, and show the count of each segment
pd.read_sql("""
with customer_type as(
select customer_id, previous_purchases,
case 
	when previous_purchases = 1 then 'New'
	when previous_purchases BETWEEN 2 and 10 then 'Returning'
	else 'Loyal'
	end as customer_segment
from customer
)
select customer_segment, count(*) as "Number of Customers"
from customer_type
group by customer_segment
""",conn)

,customer_segment,Number of Customers
0,Loyal,3116
1,New,83
2,Returning,701


In [25]:
# Top 3 most purchased products with each category
pd.read_sql("""
with item_counts as (
select category,
item_purchased,
count(customer_id) as total_orders,
row_number() over(partition by category order by count(customer_id) desc) as item_rank
from customer
group by category, item_purchased
)

select item_rank, category, item_purchased, total_orders
from item_counts
where item_rank <= 3;
""",conn)

,item_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Pants,171
4,2,Clothing,Blouse,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


In [26]:
# Subscription status of repeat customer with 5 or more purchases
pd.read_sql("""
select subscription_status, 
count(customer_id) as repeat_buyers
from customer
where previous_purchases >= 5
group by subscription_status
""",conn)

,subscription_status,repeat_buyers
0,No,2583
1,Yes,980


In [27]:
# Revenue contribution by age group
pd.read_sql("""
select age_group, 
sum(purchase_amount) as total_rev
from customer
group by age_group
order by total_rev desc
""",conn)

,age_group,total_rev
0,Young Adult,62143
1,Middle-aged,59197
2,Adult,55978
3,Senior,55763
